In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

In [ ]:
# Custom functions
from useful_functions import get_inventory_dataset, init_simple_lca, multi_lcia, multi_contribution_analysis, calculate_projected_impacts
from visualisation_functions import plot_multilca_impacts, plot_multilca_impacts_multidb, plot_contribution_analysis, plot_production_impacts, plot_incremental_impacts, plot_iwplus_contributions, plot_scenario_production_comparison

# Set projects, LCI and LCIA methods

In [ ]:
BW_PROJECT = 'lib_rm' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

In [ ]:
EI_DB = 'ecoinvent-3.10-cutoff'
LIB_RM_DB = 'LIB raw materials'
image_2020 = 'image_26_2020'
image_2030 = 'image_26_2030'
image_2040 = 'image_26_2040'
image_2050 = 'image_26_2050'
prospective_db = [image_2020, image_2030, image_2040, image_2050]

## Import LCI

In [ ]:
INVENTORIES = {
    ## Neodymium
    "Neodymium":        ("rare earth oxides production, from rare earth carbonate concentrate", "neodymium oxide", "RoW"),
    
    ## Copper
    "Copper concentrate, Canada": ('copper mine operation and beneficiation, sulfide ore', 'copper concentrate, sulfide ore', 'CA'),
    "Market for copper, cathode": ('market for copper, cathode', 'copper, cathode', 'GLO'),
    
    ## Istrate et al (2024)
    "Lithium hydroxide, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
    "Lithium hydroxide, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
    "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
    "Graphite, synthetic":   ("graphite powder coating", "synthetic graphite, battery grade", "CN"),
    
    "Lithium carbonate, brine":        ("lithium carbonate production, Salar de Atacama", "lithium carbonate, battery grade", "CL"),
    "Lithium carbonate, spodumene":    ("lithium carbonate production, from Australian spodumene", "lithium carbonate, battery grade", "CN"),
}

## Pick LCIA methods

In [ ]:
# We can also import some from IW+2.1
iw_ei310 = 'data/IW+2.1/impact_world_plus_21_brightway2_expert_version_ei310.5535d12bedce3770ffef004e84229fd1.bw2package'
bw.BW2Package.import_file(iw_ei310)

In [ ]:
# Filter and display methods that contain "IMPACT World+" in their names
iw_methods = [method for method in bd.methods if "impact world+" in " ".join(method).lower()]
df_iw_methods = pd.DataFrame(iw_methods, columns=["Method", "Impact Category", "Subcategory"])
#df_iw_methods.to_csv(r'data/iw_methods.csv', index=False)

In [ ]:
ipcc_methods = [method for method in bd.methods if "ipcc 2021" in " ".join(method).lower()]
ipcc_methods

In [ ]:
# To take them all
IMPACT_METHODS_ALL = {method[-1]: method for method in iw_methods}
IMPACT_METHODS_ALL

In [ ]:
# We take only a few one for testing
IMPACT_METHODS = {
'Carbon footprint': ('IMPACT World+ Footprint 2.1 for ecoinvent v3.10','Carbon footprint'), 
'Total human health': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Human health','Total human health'), 
'Total ecosystem quality': ('IMPACT World+ Damage 2.1 for ecoinvent v3.10','Ecosystem quality','Total ecosystem quality'), 
}


In [ ]:
IMPACT_METHODS = {
    'GWP100': ('IPCC 2021 no LT', 'climate change no LT','global warming potential (GWP100) no LT'),
    'GTP100': ('IPCC 2021 no LT','climate change no LT', 'global temperature change potential (GTP100) no LT')
}

# Calculate specific impacts (e.g. per mass)

In [ ]:
# Initialize the dictionary to store results for each database
specific_lca_results = {}

# Iterate over each database
for db_name in prospective_db:
    # Get inventory datasets from the specific database
    INVENTORIES_ds = get_inventory_dataset(INVENTORIES, [db_name])  # Pass as a list to search only in the current db

    # Dictionary to store LCA results for each raw material in the current database
    specific_lca = {}

    # Check if INVENTORIES_ds has results for the current database
    if INVENTORIES_ds:
        for rm, activity in INVENTORIES_ds.items():
            # Dictionary to store impacts for the current raw material
            impacts = {}
            
            # Loop over each impact method in IMPACT_METHODS
            for impact_name, method_tuple in IMPACT_METHODS.items():
                try:
                    # Initialize the LCA object for the current database, inventory, and method
                    lca = init_simple_lca(activity, method=method_tuple)
                    
                    # Calculate the impact score for the current method
                    lca.lcia()  # Perform the life cycle impact assessment step
                    impacts[impact_name] = lca.score  # Store the score for this method

                except Exception as e:
                    print(f"Error with {impact_name} in {db_name} for {rm}: {e}")
            
            # Store the impacts for the current raw material
            specific_lca[rm] = impacts

        # Convert the specific LCA results for the current database to a DataFrame and store it
        specific_lca_results[db_name] = pd.DataFrame(specific_lca).T
    else:
        print(f"No inventory data found for database: {db_name}")


In [ ]:
specific_lca_results[image_2020]

In [ ]:
db_colors = {
    'image_26_2020': '#04151f',
    'image_26_2030': '#183a37',
    'image_26_2040': '#efd6ac',
    'image_26_2050': '#c44900'
}

plot_multilca_impacts_multidb(specific_lca_results, db_colors=db_colors, save_path="results/plca_comparison_plot.png")

In [ ]:
# Initialize a dictionary to store contribution analysis results by database
all_contribution_results = {}

# Loop through each database
for db_name in prospective_db:
    # Get inventory datasets from the specific database
    INVENTORIES_ds = get_inventory_dataset(INVENTORIES, [db_name])

    # Dictionary to store contribution results for each inventory in the current database
    contribution_results = {}

    # Perform contribution analysis for each inventory in the current database
    if INVENTORIES_ds:
        for rm_name, rm_ds in INVENTORIES_ds.items():
            # Initialize the LCA object
            lca = init_simple_lca(rm_ds)

            # Perform the contribution analysis for the current inventory
            contributions = multi_contribution_analysis(lca, IMPACT_METHODS, top_n=10, threshold=0.01)
            contribution_results[rm_name] = contributions

        # Store the contribution results for the current database
        all_contribution_results[db_name] = contribution_results
    else:
        print(f"No inventory data found for database: {db_name}")

# Convert the results into DataFrames for easy analysis
contribution_dfs = {}

# Loop through each database and organize the results
for db_name, db_contributions in all_contribution_results.items():
    for rm_name, impacts in db_contributions.items():
        for impact_name, contributions in impacts.items():
            df = pd.DataFrame(contributions)
            df["Database"] = db_name
            df["Inventory"] = rm_name
            df["Impact Category"] = impact_name
            contribution_dfs[(db_name, rm_name, impact_name)] = df

# Combine all individual DataFrames into one for easy viewing and analysis
contribution_analysis_df = pd.concat(contribution_dfs.values(), ignore_index=True)

# Set "Database", "Inventory", and "Impact Category" as the row indices
contribution_analysis_df.set_index(["Database", "Inventory", "Impact Category"], inplace=True)

# Display the final DataFrame
contribution_analysis_df.head()


# Calculate production-related impacts (e.g. scaled with scenarios)

## Import scenarios

They are from the Canadian Climate Institute and can be found [here](https://440megatonnes.ca/insight/canada-critical-minerals-clean-energy-transition/)

In [ ]:
production_existing = pd.read_excel(r'data/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_existing')
production_potential = pd.read_excel(r'data/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_potential')
production_existing = production_existing[production_existing['Scenario']=='Domestic demand scenario']
production_potential = production_potential[production_potential['Scenario']=='Domestic demand scenario']

In [ ]:
plot_scenario_production_comparison(production_existing, production_potential, save_path='results/demand_lca_results/cci_production_scenarios.png')

## Choose which LCI to associate with each mineral 

In [ ]:
specific_lca = specific_lca.reset_index().rename(columns={'index': 'Mineral'})
specific_lca

In [ ]:
# Define a manual mapping to link the mineral in the scenario with the LCI of our choice
mineral_to_material = {
    "Cobalt": "Cobalt",
    "Copper": "Copper concentrate, Canada",
    "Lithium": "Lithium carbonate, spodumene", 
    "Nickel": "Nickel",
    "Graphite": "Graphite, natural",
    "Neodymium": "Neodymium"  
}


## Calculate the production-related impacts of scenarios

In [ ]:
# Generate the projected impacts DataFrame
projected_impacts_existing_production = calculate_projected_impacts(production_existing, specific_lca, mineral_to_material)
projected_impacts_potential_production = calculate_projected_impacts(production_potential, specific_lca, mineral_to_material)


In [ ]:
projected_impacts_existing_production

In [ ]:
projected_impacts_existing_production.columns

In [ ]:
impact_categories = [
'Climate change ST (kg CO2 eq (short))',
'Total human health (DALY)',
'Total ecosystem quality (PDF.m2.yr)'    
]

In [ ]:
# Plot impacts per production scenario
plot_production_impacts(projected_impacts_existing_production, production_existing, 
                             impact_categories, 
                             scenario_name='existing_production')
plot_production_impacts(projected_impacts_potential_production, production_potential, 
                             impact_categories, scenario_name='potential_production')


In [ ]:
# Plot incremental impacts from potential production to existing production
plot_incremental_impacts(projected_impacts_existing_production, projected_impacts_potential_production,
                                 production_existing, production_potential, 
                                 impact_categories, save_dir="results/demand_lca_results", scenario_name="incremental_comparison")
